In [ ]:
!pip install gradio runorm transformers torch torchaudio huggingface_hub soundfile pydub silero-stress git+https://github.com/mikhail2013ru/F5-TTS-RUS.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# НАЖАТЬ RESTART SESSION

In [ ]:
#!/usr/bin/env python3
import time
import re
import os
import gc
import tempfile
import traceback
import json
import threading
from pydub import AudioSegment
from pathlib import Path

import gradio as gr
import numpy as np
import soundfile as sf
import torch
import torchaudio

from huggingface_hub import login, hf_hub_download, snapshot_download

import gradio.blocks
import gradio.components

# ================ УЛУЧШЕННАЯ АВТОРИЗАЦИЯ HUGGINGFACE ================
def setup_huggingface_auth():
    """Настройка авторизации HuggingFace для доступа к закрытым репозиториям"""
    print("\n" + "="*60)
    print("🔐 НАСТРОЙКА АВТОРИЗАЦИИ HUGGINGFACE")
    print("="*60)
    
    # Проверяем разные способы получения токена в порядке приоритета
    hf_token = None
    token_source = "Не найден"
    
    # 1. Проверяем переменную окружения
    if os.environ.get("HF_TOKEN"):
        hf_token = os.environ["HF_TOKEN"]
        token_source = "переменная окружения HF_TOKEN"
    
    # 2. Проверяем файл с токеном
    elif os.path.exists("/root/.cache/huggingface/token"):
        try:
            with open("/root/.cache/huggingface/token", "r") as f:
                hf_token = f.read().strip()
            if hf_token:
                token_source = "файл /root/.cache/huggingface/token"
        except Exception as e:
            print(f"⚠️ Ошибка чтения файла токена: {e}")
    
    # 3. Если в Colab, проверяем секреты
    else:
        try:
            from google.colab import userdata
            hf_token = userdata.get('HF_TOKEN')
            if hf_token:
                token_source = "секреты Colab"
        except ImportError:
            print("⚠️ Не в Colab или google.colab недоступен")
        except Exception as e:
            print(f"⚠️ Ошибка получения токена из Colab: {e}")
    
    # 4. Проверяем другие возможные места
    if not hf_token:
        # Проверяем стандартный файл Hugging Face
        hf_cache_token = os.path.expanduser("~/.cache/huggingface/token")
        if os.path.exists(hf_cache_token):
            try:
                with open(hf_cache_token, "r") as f:
                    hf_token = f.read().strip()
                if hf_token:
                    token_source = f"файл {hf_cache_token}"
            except:
                pass
    
    # Если токен найден, авторизуемся
    if hf_token:
        try:
            print(f"✅ Токен найден в {token_source}")
            
            # Проверяем формат токена
            if not hf_token.startswith("hf_"):
                print(f"⚠️ Предупреждение: Токен может быть неверного формата. Должен начинаться с 'hf_'")
            
            # Пробуем авторизоваться
            login(token=hf_token)
            
            # Сохраняем токен в переменную окружения
            os.environ["HF_TOKEN"] = hf_token
            
            # Сохраняем токен в файл для будущих сессий
            try:
                # Создаем директорию если её нет
                os.makedirs("/root/.cache/huggingface", exist_ok=True)
                with open("/root/.cache/huggingface/token", "w") as f:
                    f.write(hf_token)
                print("✅ Токен сохранен для будущих запусков")
            except Exception as e:
                print(f"⚠️ Не удалось сохранить токен в файл: {e}")
            
            print("✅ Авторизация HuggingFace успешна!")
            return hf_token
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ Ошибка авторизации: {error_msg}")
            
            # Проверяем специфические ошибки
            if "401" in error_msg:
                print("❌ Неверный токен авторизации. Проверьте правильность токена.")
            elif "400" in error_msg:
                print("❌ Неверный формат токена. Токен должен начинаться с 'hf_'.")
            elif "connection" in error_msg.lower():
                print("❌ Проблемы с подключением к интернету. Проверьте соединение.")
            
            return None
    else:
        print("ℹ️  Токен не найден. Продолжаем без авторизации.")
        print("   Закрытые репозитории будут недоступны.")
        
        # Создаем информационное сообщение
        print("\n" + "-"*60)
        print("📋 Как добавить токен:")
        print("1. Получите токен на https://huggingface.co/settings/tokens")
        print("2. Добавьте токен одним из способов:")
        print("   • В Colab: добавьте в секреты с ключом 'HF_TOKEN'")
        print("   • В переменные окружения: export HF_TOKEN='ваш_токен'")
        print("   • В файл: echo 'ваш_токен' > /root/.cache/huggingface/token")
        print("-"*60)
        
        return None

# Вызываем авторизацию в начале
print("\n" + "="*60)
print("🚀 ЗАПУСК ESpeech-TTS")
print("="*60)

HF_TOKEN = setup_huggingface_auth()
print("="*60 + "\n")

# ================ ГЛОБАЛЬНЫЕ ФЛАГИ ДЛЯ УПРАВЛЕНИЯ ГЕНЕРАЦИЕЙ ================
stop_generation = False
generation_thread = None
generation_event = threading.Event()
stop_requested = False

from silero_stress import load_accentor
from runorm import RUNorm
from f5_tts.infer.utils_infer import (
    infer_process,
    load_model,
    load_vocoder,
    preprocess_ref_audio_text,
    remove_silence_for_generated_wav,
    save_spectrogram,
    tempfile_kwargs,
)
from f5_tts.model import DiT

# 🔹 Добавьте эти импорты для Whisper
from transformers import WhisperProcessor, WhisperForConditionalGeneration

MODELS_CONFIG = {
    "ESpeech-TTS-1_SFT-95K": {
        "repo": "ESpeech/ESpeech-TTS-1_SFT-95K",
        "model_file": "espeech_tts_95k.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "ESpeech-TTS-1_RL-V1": {
        "repo": "ESpeech/ESpeech-TTS-1_RL-V1",
        "model_file": "espeech_tts_rlv1.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "ESpeech-TTS-1_RL-V2": {
        "repo": "ESpeech/ESpeech-TTS-1_RL-V2",
        "model_file": "espeech_tts_rlv2.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_v2_Misha24-10": {
        "repo": "Misha24-10/F5-TTS_RUSSIAN",
        "model_file": "F5TTS_v1_Base_v2/model_last_inference.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_v4_winter": {
        "repo": "Misha24-10/F5-TTS_RUSSIAN",
        "model_file": "F5TTS_v1_Base_v4_winter/model_212000.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_1.25M_SWivid": {
        "repo": "SWivid/F5-TTS",
        "model_file": "F5TTS_v1_Base/model_1250000.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
}

# Глобальные переменные для моделей
loaded_models = {}
loaded_vocoder = None
current_model_name = None
remember_seed = False
last_seed = -1

# 🔴 НОВОЕ: Словарь для локальных моделей
local_models_config = {}

# 🔴 НОВОЕ: Словарь для локальных моделей и файл для сохранения
LOCAL_MODELS_FILE = "local_models.json"
local_models_config = {}

def load_local_models():
    """Загрузка локальных моделей из файла"""
    global local_models_config
    try:
        if os.path.exists(LOCAL_MODELS_FILE):
            with open(LOCAL_MODELS_FILE, 'r', encoding='utf-8') as f:
                local_models_config = json.load(f)
            print(f"✅ Загружено {len(local_models_config)} локальных моделей")
        else:
            local_models_config = {}
            print("✅ Файл локальных моделей не найден, создан новый")
    except Exception as e:
        print(f"❌ Ошибка загрузки локальных моделей: {e}")
        local_models_config = {}

def save_local_models():
    """Сохранение локальных моделей в файл"""
    try:
        with open(LOCAL_MODELS_FILE, 'w', encoding='utf-8') as f:
            json.dump(local_models_config, f, ensure_ascii=False, indent=2)
        print(f"✅ Сохранено {len(local_models_config)} локальных моделей")
    except Exception as e:
        print(f"❌ Ошибка сохранения локальных моделей: {e}")

# 🔴 Загружаем локальные модели при старте
load_local_models()

def get_all_models_config():
    """Получить объединенный конфиг моделей"""
    return {**MODELS_CONFIG, **local_models_config}

def clear_loaded_models():
    """Очистка загруженных моделей из памяти И удаление локальных моделей из дропбокса"""
    global loaded_models, loaded_vocoder, current_model_name, local_models_config
    
    # Сохраняем список удаляемых локальных моделей для сообщения
    removed_local_models = list(local_models_config.keys())
    
    # Очищаем модели из памяти
    loaded_models.clear()
    loaded_vocoder = None
    current_model_name = None
    
    # 🔴 ОЧИЩАЕМ ТОЛЬКО ЛОКАЛЬНЫЕ МОДЕЛИ ИЗ КОНФИГУРАЦИИ (УДАЛЯЕМ ИЗ ДРОПБОКСА)
    local_models_config.clear()
    save_local_models()
    
    # Очищаем кэш CUDA
    torch.cuda.empty_cache()
    gc.collect()
    
    # Обновляем общий конфиг моделей (теперь только стандартные модели)
    updated_models_config = get_all_models_config()  # 🔴 Теперь вернет только MODELS_CONFIG
    
    # Выбираем первую доступную СТАНДАРТНУЮ модель
    new_model = list(updated_models_config.keys())[0] if updated_models_config else None
    
    removed_count = len(removed_local_models)
    status_message = f"✅ Удалено {removed_count} локальных моделей и выгружено из памяти"
    
    print(status_message)
    
    return (
        gr.update(value=f"<div class='model-status model-loaded'>{status_message}</div>", visible=True),
        gr.update(value=""),  # Очищаем поле пути к модели
        gr.update(value=""),  # Очищаем поле пути к словарю
        gr.update(value=""),  # Очищаем поле имени модели
        gr.update(choices=list(updated_models_config.keys()), value=new_model)  # 🔴 ОБНОВЛЯЕМ ДРОПБОКС (только стандартные модели)
    )

def remove_local_model(model_name):
    """Удаление локальной модели из конфигурации"""
    global local_models_config
    
    if model_name in local_models_config:
        del local_models_config[model_name]
        save_local_models()
        
        # Обновляем общий конфиг моделей
        updated_models_config = get_all_models_config()
        
        # Выбираем первую доступную модель
        new_model = list(updated_models_config.keys())[0] if updated_models_config else None
        
        return (
            gr.update(choices=list(updated_models_config.keys()), value=new_model),
            gr.update(value=f"✅ Модель '{model_name}' удалена", visible=True)
        )
    else:
        return gr.update(), gr.update(value=f"❌ Модель '{model_name}' не найдена", visible=True)

def check_cuda_availability():
    """Check if CUDA is available and raise error if not"""
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available! This application requires GPU with CUDA support.")
    print(f"CUDA is available. Using device: {torch.cuda.get_device_name()}")

def parse_hf_path(hf_path):
    """Парсит HF путь в repo_id и filename"""
    if hf_path.startswith('hf://'):
        path_parts = hf_path.replace('hf://', '').split('/')
        if len(path_parts) >= 2:
            # Репо состоит из первых двух частей: namespace/repo_name
            repo_id = '/'.join(path_parts[:2])  # Только первые два элемента
            # Файл - все остальные части
            filename = '/'.join(path_parts[2:])  # Все начиная с третьего элемента
            return repo_id, filename
    return None, None

def add_local_model(model_path, vocab_path, model_name):
    """Добавить локальную модель в конфигурацию"""
    global local_models_config, MODELS_CONFIG
    
    if not model_path or not vocab_path or not model_name:
        return gr.update(), gr.update(value="❌ Заполните все поля", visible=True)
    
    # 🔴 ИСПРАВЛЕННАЯ ПРОВЕРКА HF ПУТЕЙ
    def validate_hf_path(file_path, file_type):
        if file_path.startswith('hf://'):
            repo_id, filename = parse_hf_path(file_path)
            if not repo_id or not filename:
                return False, f"❌ Неверный формат HF пути для {file_type}"
            
            # Дополнительная проверка формата repo_id
            if '/' not in repo_id or len(repo_id.split('/')) != 2:
                return False, f"❌ Неверный формат repo_id: {repo_id}"
                
            return True, None
        else:
            # Для локальных путей проверяем существование файла
            if os.path.exists(file_path):
                return True, None
            else:
                return False, f"❌ Файл {file_type} не найден: {file_path}"
    
    # Проверяем путь к модели
    is_model_valid, model_error = validate_hf_path(model_path, "модели")
    if not is_model_valid:
        return gr.update(), gr.update(value=model_error, visible=True)
    
    # Проверяем путь к словарю
    is_vocab_valid, vocab_error = validate_hf_path(vocab_path, "словаря")
    if not is_vocab_valid:
        return gr.update(), gr.update(value=vocab_error, visible=True)
    
    # Добавляем модель в локальную конфигурацию
    local_models_config[model_name] = {
        "repo": "local",
        "model_file": model_path,
        "vocab_file": vocab_path,
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    }

    # 🔴 СОХРАНЯЕМ В ФАЙЛ
    save_local_models()
    
    # Обновляем общий конфиг моделей
    updated_models_config = {**MODELS_CONFIG, **local_models_config}
    
    # Возвращаем ОБА значения: обновленный dropdown и сообщение статуса
    return gr.update(
        choices=list(updated_models_config.keys()),
        value=model_name
    ), gr.update(value=f"✅ Локальная модель '{model_name}' добавлена", visible=True)

# 🔴 ДОБАВЛЕНО: Функция для автоматической проверки токена
def check_token_validity(token):
    """Проверить валидность токена"""
    if not token:
        return False, "Токен пустой"
    
    # Базовая проверка формата
    if not token.startswith("hf_"):
        return False, "Токен должен начинаться с 'hf_'"
    
    if len(token) < 20:
        return False, "Токен слишком короткий"
    
    return True, "Формат токена корректный"

# 🔴 ОБНОВЛЕННАЯ ФУНКЦИЯ: Обновление токена HuggingFace
def update_hf_token(new_token):
    """Обновление токена HuggingFace"""
    global HF_TOKEN
    
    if not new_token:
        return gr.update(value="❌ Введите токен", visible=True)
    
    # Проверяем валидность токена
    is_valid, validation_msg = check_token_validity(new_token)
    if not is_valid:
        return gr.update(value=f"❌ {validation_msg}", visible=True)
    
    try:
        print(f"🔄 Попытка авторизации с новым токеном...")
        
        # Пробуем авторизоваться
        login(token=new_token)
        HF_TOKEN = new_token
        
        # Сохраняем токен для будущих сессий
        try:
            # Создаем директорию если её нет
            os.makedirs("/root/.cache/huggingface", exist_ok=True)
            
            # Сохраняем в файл
            with open("/root/.cache/huggingface/token", "w") as f:
                f.write(new_token)
            print(f"✅ Токен сохранен в файл")
        except Exception as file_error:
            print(f"⚠️ Не удалось сохранить токен в файл: {file_error}")
        
        # Сохраняем в переменную окружения
        os.environ["HF_TOKEN"] = new_token
        
        print(f"✅ Токен обновлен")
        return gr.update(value="✅ Токен успешно обновлен и сохранен!", visible=True)
    
    except Exception as e:
        error_msg = str(e)
        print(f"❌ Ошибка обновления токена: {error_msg}")
        
        # Более подробные сообщения об ошибках
        if "401" in error_msg:
            detailed_msg = "❌ Неверный токен авторизации. Проверьте правильность токена."
        elif "400" in error_msg:
            detailed_msg = "❌ Неверный формат токена."
        elif "connection" in error_msg.lower():
            detailed_msg = "❌ Проблемы с подключением к интернету. Проверьте соединение."
        else:
            detailed_msg = f"❌ Ошибка: {str(e)[:100]}"
        
        return gr.update(value=detailed_msg, visible=True)

def add_local_model_with_auth(model_path, vocab_path, model_name, hf_token=None):
    """Добавить локальную модель с поддержкой авторизации"""
    global local_models_config, MODELS_CONFIG, HF_TOKEN
    
    if not model_path or not vocab_path or not model_name:
        return gr.update(), gr.update(value="❌ Заполните все поля", visible=True)
    
    # Используем переданный токен или глобальный
    use_token = hf_token if hf_token else HF_TOKEN
    
    # 🔴 ИСПРАВЛЕННАЯ ПРОВЕРКА HF ПУТЕЙ
    def validate_hf_path(file_path, file_type):
        if file_path.startswith('hf://'):
            repo_id, filename = parse_hf_path(file_path)
            if not repo_id or not filename:
                return False, f"❌ Неверный формат HF пути для {file_type}"
            
            # Дополнительная проверка формата repo_id
            if '/' not in repo_id or len(repo_id.split('/')) != 2:
                return False, f"❌ Неверный формат repo_id: {repo_id}"
                
            return True, None
        else:
            # Для локальных путей проверяем существование файла
            if os.path.exists(file_path):
                return True, None
            else:
                return False, f"❌ Файл {file_type} не найден: {file_path}"
    
    # Проверяем путь к модели
    is_model_valid, model_error = validate_hf_path(model_path, "модели")
    if not is_model_valid:
        return gr.update(), gr.update(value=model_error, visible=True)
    
    # Проверяем путь к словарю
    is_vocab_valid, vocab_error = validate_hf_path(vocab_path, "словаря")
    if not is_vocab_valid:
        return gr.update(), gr.update(value=vocab_error, visible=True)
    
    # 🔴 ФУНКЦИЯ ДЛЯ СКАЧИВАНИЯ С ТОКЕНОМ
    def download_hf_file_with_token(hf_path, file_type):
        if hf_path.startswith('hf://'):
            try:
                repo_id, filename = parse_hf_path(hf_path)
                if not repo_id or not filename:
                    raise ValueError(f"Invalid HF path: {hf_path}")
                
                print(f"Скачивание {file_type} из {repo_id}/{filename}")
                
                # Скачиваем с токеном
                downloaded_path = hf_hub_download(
                    repo_id=repo_id,
                    filename=filename,
                    repo_type="model",
                    token=use_token  # ⬅️ ИСПОЛЬЗУЕМ ТОКЕН
                )
                print(f"✅ {file_type} скачан: {downloaded_path}")
                return downloaded_path
                
            except Exception as e:
                error_msg = str(e)
                if "401" in error_msg or "403" in error_msg:
                    if use_token is None:
                        raise PermissionError(f"❌ Репозиторий {repo_id} приватный. Требуется токен авторизации!")
                    else:
                        raise PermissionError(f"❌ Нет доступа к {repo_id}. Проверьте токен или права доступа.")
                elif "404" in error_msg:
                    raise FileNotFoundError(f"❌ Файл {filename} не найден в репозитории {repo_id}")
                else:
                    raise e
        else:
            # Локальный путь
            if not os.path.exists(hf_path):
                raise FileNotFoundError(f"Файл не найден: {hf_path}")
            return hf_path
    
    try:
        # Скачиваем файлы с токеном
        downloaded_model = download_hf_file_with_token(model_path, "модели")
        downloaded_vocab = download_hf_file_with_token(vocab_path, "словаря")
        
        # Добавляем в конфиг
        local_models_config[model_name] = {
            "repo": "local",
            "model_file": downloaded_model,  # Сохраняем локальный путь
            "vocab_file": downloaded_vocab,
            "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
        }

        # 🔴 СОХРАНЯЕМ В ФАЙЛ
        save_local_models()
        
        # Обновляем общий конфиг моделей
        updated_models_config = {**MODELS_CONFIG, **local_models_config}
        
        # Возвращаем ОБА значения: обновленный dropdown и сообщение статуса
        return gr.update(
            choices=list(updated_models_config.keys()),
            value=model_name
        ), gr.update(value=f"✅ Локальная модель '{model_name}' добавлена", visible=True)
        
    except Exception as e:
        error_msg = f"❌ Ошибка: {str(e)}"
        print(f"Ошибка добавления модели: {traceback.format_exc()}")
        return gr.update(), gr.update(value=error_msg, visible=True)

def load_model_with_progress(model_name, progress_callback=None):
    """Загрузка модели с индикатором прогресса"""
    global loaded_models, current_model_name, stop_generation
    
    # Проверка флага остановки
    if stop_generation:
        print("🛑 Загрузка остановлена по запросу пользователя")
        return None
    
    # Объединяем конфиги моделей
    all_models_config = get_all_models_config()
    
    # Валидация входных параметров
    if not model_name or model_name not in all_models_config:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Если модель уже загружена, возвращаем её
    if model_name in loaded_models:
        print(f"✅ Модель '{model_name}' уже загружена")
        return loaded_models[model_name]
    
    # Получаем конфигурацию модели
    config = all_models_config[model_name]
    
    # Загрузка модели
    print(f"📥 Загрузка модели: {model_name}")
    
    # 🔴 ВЫНЕСЕНА ФУНКЦИЯ download_hf_file ЗА ПРЕДЕЛЫ БЛОКОВ
    def download_hf_file(hf_path, file_type, progress_stage):
        """Скачивает файл из HF если путь начинается с hf://"""
        if hf_path.startswith('hf://'):
            if progress_callback:
                progress_callback(f"📥 Загрузка {file_type} из Hugging Face...", progress_stage)
            
            try:
                # 🔴 ИСПРАВЛЕННЫЙ ПАРСИНГ: repo_id = первые 2 части, filename = остальные
                repo_id, filename = parse_hf_path(hf_path)
                if not repo_id or not filename:
                    raise ValueError(f"Invalid HF path format: {hf_path}")
                
                print(f"Downloading {file_type} from HF: {repo_id}/{filename}")

                downloaded_path = hf_hub_download(
                    repo_id=repo_id, 
                    filename=filename,
                    repo_type="model",
                    token=HF_TOKEN  # ⬅️ ДОБАВЛЯЕМ ТОКЕН ДЛЯ ЗАКРЫТЫХ РЕПОЗИТОРИЕВ
                )
                print(f"✅ {file_type} downloaded to: {downloaded_path}")
                return downloaded_path
                
            except Exception as e:
                print(f"❌ Failed to download {file_type} from HF: {e}")
                # Детализация ошибок
                if "404" in str(e):
                    raise FileNotFoundError(f"Файл {filename} не найден в репозитории {repo_id}")
                elif "401" in str(e):
                    # 🔴 УЛУЧШЕННОЕ СООБЩЕНИЕ ДЛЯ ЗАКРЫТЫХ РЕПОЗИТОРИЕВ
                    if HF_TOKEN is None:
                        raise PermissionError(f"Нет доступа к репозиторию {repo_id}. Требуется авторизация!")
                    else:
                        raise PermissionError(f"Нет доступа к репозиторию {repo_id}. Проверьте токен и права доступа.")
                else:
                    raise e
        else:
            # Локальный путь - проверяем существование
            if not os.path.exists(hf_path):
                raise FileNotFoundError(f"{file_type} file not found: {hf_path}")
            return hf_path
    
    # 🔴 ОБНОВЛЕННАЯ ЛОГИКА: ОБРАБОТКА ЛОКАЛЬНЫХ И HF ПУТЕЙ
    if config['repo'] == "local":
        if progress_callback:
            progress_callback("🔍 Обработка локальной модели...", 0.1)

        # 🔴 ПРОВЕРКА ОСТАНОВКИ
        if stop_generation:
            print("🛑 Остановка на этапе поиска модели")
            return None
        
        model_path = config['model_file']
        vocab_path = config['vocab_file']
        
        # Скачиваем/проверяем файлы модели и словаря
        try:
            model_path = download_hf_file(model_path, "model", 0.2)
            vocab_path = download_hf_file(vocab_path, "vocab", 0.3)
            
        except Exception as e:
            error_msg = f"❌ Ошибка загрузки файлов для локальной модели: {str(e)}"
            print(error_msg)
            raise e
            
        print(f"📁 Loading local model from: {model_path}")
        
    else:
        # 🔴 СТАНДАРТНАЯ ЛОГИКА ДЛЯ МОДЕЛЕЙ ИЗ HUGGINGFACE
        if progress_callback:
            progress_callback("🔍 Поиск модели в HuggingFace...", 0.1)
        
        # 🔴 ПРОВЕРКА ОСТАНОВКИ
        if stop_generation:
            print("🛑 Остановка на этапе поиска модели в HF")
            return None
        
        # Проверяем доступность CUDA
        check_cuda_availability()

        # 🔴 ПРОВЕРКА ОСТАНОВКИ ПОСЛЕ ПРОВЕРКИ CUDA
        if stop_generation:
            print("🛑 Остановка после проверки CUDA")
            return None
        
        # 🔴 ЗАГРУЗКА ФАЙЛОВ ДЛЯ HF МОДЕЛЕЙ
        if progress_callback:
            progress_callback("📥 Загрузка модели...", 0.3)
        
        # 🔴 ПРОВЕРКА ОСТАНОВКИ ПЕРЕД ЗАГРУЗКОЙ
        if stop_generation:
            print("🛑 Остановка перед загрузкой файлов")
            return None
        
        try:
            model_path = hf_hub_download(
                repo_id=config['repo'], 
                filename=config['model_file'],
                repo_type="model"
            )
            vocab_path = hf_hub_download(
                repo_id=config['repo'], 
                filename=config['vocab_file'],
                repo_type="model"
            )
            print(f"✅ Model downloaded to: {model_path}")
            print(f"✅ Vocab downloaded to: {vocab_path}")
            
        except Exception as e:
            print("❌ hf_hub_download failed:", e)
            
            # 🔴 РЕЗЕРВНАЯ СТРАТЕГИЯ: скачивание всей модели
            if progress_callback:
                progress_callback("📦 Резервное скачивание полной модели...", 0.5)
            
            # 🔴 ПРОВЕРКА ОСТАНОВКИ ПЕРЕД РЕЗЕРВНОЙ СТРАТЕГИЕЙ
            if stop_generation:
                print("🛑 Остановка перед резервной стратегией")
                return None
            
            try:
                local_dir = f"cache_{config['repo'].replace('/', '_')}"
                snapshot_dir = snapshot_download(
                    repo_id=config['repo'], 
                    cache_dir=None, 
                    local_dir=local_dir,
                    repo_type="model",
                    allow_patterns=[config['model_file'], config['vocab_file']]
                )
                
                # Ищем файлы в скачанной директории
                possible_model = os.path.join(snapshot_dir, config['model_file'])
                possible_vocab = os.path.join(snapshot_dir, config['vocab_file'])
                
                if os.path.exists(possible_model):
                    model_path = possible_model
                    print(f"✅ Model found in snapshot: {model_path}")
                else:
                    print(f"❌ Model file not found in snapshot: {possible_model}")
                    
                if os.path.exists(possible_vocab):
                    vocab_path = possible_vocab
                    print(f"✅ Vocab found in snapshot: {vocab_path}")
                else:
                    print(f"❌ Vocab file not found in snapshot: {possible_vocab}")
                    
            except Exception as snapshot_error:
                print("❌ snapshot_download also failed:", snapshot_error)
                # Комбинируем ошибки для лучшей диагностики
                combined_error = f"Original error: {e}\nSnapshot error: {snapshot_error}"
                raise Exception(f"Failed to download model: {combined_error}")
    
    # 🔴 ПРОВЕРКА ФЛАГА ОСТАНОВКИ ПОСЛЕ ЗАГРУЗКИ ФАЙЛОВ
    if stop_generation:
        print("🛑 Generation stopped during file download")
        return None
    
    # 🔴 ФИНАЛЬНАЯ ПРОВЕРКА СУЩЕСТВОВАНИЯ ФАЙЛОВ
    if not model_path or not os.path.exists(model_path):
        raise FileNotFoundError(f"❌ Model file not found: {model_path}")
    if not vocab_path or not os.path.exists(vocab_path):
        raise FileNotFoundError(f"❌ Vocab file not found: {vocab_path}")
    
    # 🔴 ПРОВЕРКА ОСТАНОВКИ ПЕРЕД ЗАГРУЗКОЙ В ПАМЯТЬ
    if stop_generation:
        print("🛑 Остановка перед загрузкой в память")
        return None

    # 🔴 ЗАГРУЗКА МОДЕЛИ В ПАМЯТЬ
    if progress_callback:
        progress_callback("🔄 Загрузка модели в память...", 0.7)
    
    try:
        print(f"🤖 Loading model architecture from: {model_path}")
        print(f"📖 Using vocab file: {vocab_path}")
        
        model = load_model(DiT, config['model_cfg'], model_path, vocab_file=vocab_path)
        print("✅ Model architecture loaded successfully")
        
    except Exception as e:
        error_msg = f"❌ Failed to load model architecture: {str(e)}"
        print(error_msg)
        print(f"Model config: {config['model_cfg']}")
        raise e

    # 🔴 ПРОВЕРКА ФЛАГА ОСТАНОВКИ ПОСЛЕ ЗАГРУЗКИ АРХИТЕКТУРЫ
    if stop_generation:
        print("🛑 Generation stopped during model loading")
        return None
        
    # 🔴 ПРОВЕРКА ОСТАНОВКИ ПЕРЕД ПЕРЕНОСОМ НА GPU
    if stop_generation:
        print("🛑 Остановка перед переносом на GPU")
        return None
        
    # 🔴 ПЕРЕМЕЩЕНИЕ МОДЕЛИ НА GPU
    if progress_callback:
        progress_callback("🚀 Перенос модели на GPU...", 0.9)
    
    try:
        device = torch.device("cuda")
        model.to(device)
        print(f"✅ Model moved to CUDA: {device}")
        
        # Проверяем, что модель действительно на GPU
        if next(model.parameters()).is_cuda:
            print("✅ Model parameters are on GPU")
        else:
            print("⚠️ Warning: Model parameters might not be on GPU")
            
    except Exception as e:
        error_msg = f"❌ Failed to move model to GPU: {str(e)}"
        print(error_msg)
        raise e

    # 🔴 ПРОВЕРКА ОСТАНОВКИ ПЕРЕД СОХРАНЕНИЕМ В КЭШ
    if stop_generation:
        print("🛑 Остановка перед сохранением в кэш")
        return None

    # 🔴 СОХРАНЕНИЕ МОДЕЛИ В КЭШ
    loaded_models[model_name] = model
    current_model_name = model_name

    # 🔴 ФИНАЛЬНЫЙ ПРОГРЕСС И ВЫВОД
    if progress_callback:
        progress_callback("✅ Модель загружена и готова!", 1.0)
    
    print(f"🎉 Model '{model_name}' successfully loaded and cached")
    print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    return model

def ensure_model(model_name, progress_callback=None):
    """Ensure model is loaded with progress tracking"""
    global loaded_models
    
    if not model_name:
        raise ValueError("Model name must be specified")
    
    # Объединяем конфиги моделей
    all_models_config = get_all_models_config()
    
    if model_name not in all_models_config:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Если модель уже загружена, возвращаем её
    if model_name in loaded_models:
        return loaded_models[model_name]
    
    # Иначе загружаем с отслеживанием прогресса
    return load_model_with_progress(model_name, progress_callback)

def ensure_vocoder():
    global loaded_vocoder
    if loaded_vocoder is not None:
        return loaded_vocoder

    # Проверка CUDA
    check_cuda_availability()

    print("⏳ Loading vocoder...")
    
    try:
        loaded_vocoder = load_vocoder()
        device = torch.device("cuda")
        loaded_vocoder.to(device)
        print("✅ Vocoder loaded successfully")
        return loaded_vocoder
    except Exception as e:
        print(f"❌ Vocoder loading failed: {e}")
        raise e

def stop_generation_process():
    """Остановка генерации - устанавливает глобальный флаг"""
    global stop_generation, generation_event, stop_requested
    
    print("🛑 Пользователь запросил остановку генерации...")
    stop_generation = True
    stop_requested = True
    generation_event.set()  # Сигнализируем событие остановки
    
    # Пытаемся прервать поток, если он существует
    global generation_thread
    if generation_thread and generation_thread.is_alive():
        print("⚠️ Пытаемся прервать поток генерации...")
    
    # Принудительно очищаем память CUDA
    torch.cuda.empty_cache()
    gc.collect()
    
    print("✅ Флаг остановки установлен, очищена память CUDA")
    return "🛑 Запрос на остановку отправлен..."

def reset_generation_flags():
    """Сброс флагов генерации перед началом новой генерации"""
    global stop_generation, generation_event, stop_requested
    stop_generation = False
    stop_requested = False
    generation_event.clear()
    print("🔄 Флаги генерации сброшены")

def check_stop_generation():
    """Проверка флага остановки (вызывается из функций генерации)"""
    global stop_generation, generation_event
    
    # Проверяем флаг
    if stop_generation:
        print("🛑 Обнаружен запрос остановки!")
        raise KeyboardInterrupt("Генерация остановлена пользователем")
    
    # Также проверяем event для более быстрого реагирования
    if generation_event.is_set():
        print("🛑 Обнаружено событие остановки!")
        raise KeyboardInterrupt("Генерация остановлена пользователем")
    
    return False

# --- Загрузка RUNorm с правильным методом ---
print("Loading RUNorm (text normalizer)...")
try:
    normalizer = RUNorm()
    normalizer.load(
        model_size="medium",
        device="cuda" if torch.cuda.is_available() else "cpu",
        workdir="./local_cache"
    )
    
    # 🔴 ПРАВИЛЬНЫЙ МЕТОД: используем norm()
    test_text = "13,8 миллиардов лет"
    try:
        result = normalizer.norm(test_text)
        print(f"✅ RUNorm.norm SUCCESS: '{test_text}' -> '{result}'")
        print("✅ RUNorm loaded successfully with 'norm' method")
    except Exception as e:
        print(f"❌ RUNorm.norm failed: {e}")
        # Создаем заглушку при ошибке
        class DummyNormalizer:
            def norm(self, text):
                return text
        normalizer = DummyNormalizer()
        print("⚠️ Using dummy normalizer due to error")
    
except Exception as e:
    print(f"❌ Failed to load RUNorm: {e}")
    print(f"🔍 RUNorm error details: {traceback.format_exc()}")
    
    # Создаем заглушку при ошибке
    class DummyNormalizer:
        def norm(self, text):
            return text
    normalizer = DummyNormalizer()
    print("⚠️ Using dummy normalizer due to load error")

# --- Check CUDA ---
print("Loading Silero Stress model...")
accentor = load_accentor()  # ✅ Загружаем модель

# Тест accentor
test_word = "привет"
test_result = accentor(test_word)
print(f"✅ Silero Stress loaded. Тест: '{test_word}' -> '{test_result}'")

print("Loading Whisper ASR model (supports Russian and English)...")
try:
    from transformers import WhisperProcessor, WhisperForConditionalGeneration

    ASR_MODEL_NAME = "openai/whisper-medium"  # Быстро и точно. Можно: "openai/whisper-base", "medium", "large-v3"
    asr_processor = WhisperProcessor.from_pretrained(ASR_MODEL_NAME)
    asr_model = WhisperForConditionalGeneration.from_pretrained(ASR_MODEL_NAME)
    asr_model.to("cuda" if torch.cuda.is_available() else "cpu")
    print("Whisper model loaded. Supports Russian, English, and many other languages.")
except Exception as e:
    print(f"Failed to load Whisper model: {e}")
    asr_processor = None
    asr_model = None

# ================ ФУНКЦИЯ ПРОВЕРКИ АУДИО ================
def check_audio_file(audio_path):
    """Проверка аудиофайла"""
    if not audio_path:
        return gr.update(value="❌ Нет аудиофайла", visible=True)
    
    try:
        # Проверяем существование файла
        if not os.path.exists(audio_path):
            return gr.update(value="❌ Файл не существует", visible=True)
        
        # Проверяем формат
        valid_extensions = ('.wav', '.mp3', '.ogg', '.flac', '.m4a', '.aac')
        if not audio_path.lower().endswith(valid_extensions):
            return gr.update(value="❌ Неподдерживаемый формат", visible=True)
        
        # Пытаемся загрузить аудио
        try:
            audio = AudioSegment.from_file(audio_path)
            duration = len(audio) / 1000.0  # в секундах
            
            return gr.update(
                value=f"✅ Аудио файл корректен:<br>"
                      f"• Длительность: {duration:.1f} сек<br>"
                      f"• Частота: {audio.frame_rate} Гц<br>"
                      f"• Каналы: {audio.channels}<br>"
                      f"• Размер: {os.path.getsize(audio_path) / 1024:.1f} KB",
                visible=True
            )
        except Exception as e:
            return gr.update(value=f"❌ Ошибка загрузки аудио: {str(e)[:50]}", visible=True)
            
    except Exception as e:
        return gr.update(value=f"❌ Ошибка: {str(e)[:50]}", visible=True)
# ================ КОНЕЦ ФУНКЦИИ ================

# ================ НАЧАЛО ВСТАВКИ ================
# 🔴 ДОБАВЛЕНО: Тестовая функция для Whisper
def test_whisper_model():
    """Проверка работы Whisper модели перед запуском"""
    print("\n" + "="*60)
    print("🤖 ТЕСТИРОВАНИЕ WHISPER МОДЕЛИ")
    print("="*60)
    
    if asr_processor is None or asr_model is None:
        print("❌ Whisper модель не загружена")
        return False
    
    try:
        # Создаем тестовый аудиофайл (синусоида 440 Гц на 2 секунды)
        sample_rate = 16000
        duration = 2.0
        t = np.linspace(0, duration, int(sample_rate * duration))
        test_audio = 0.5 * np.sin(2 * np.pi * 440 * t)  # Нота Ля 440 Гц
        
        # Сохраняем во временный файл
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
            test_file = f.name
            sf.write(test_file, test_audio, sample_rate)
        
        print("🎵 Создан тестовый аудиофайл...")
        
        # Пробуем транскрибировать
        transcription = transcribe_audio(test_file)
        
        # Удаляем временный файл
        try:
            os.unlink(test_file)
        except:
            pass
        
        if transcription:
            print(f"✅ Whisper работает! Тестовая транскрипция: '{transcription}'")
            return True
        else:
            print("❌ Whisper вернул пустую транскрипция")
            return False
            
    except Exception as e:
        print(f"❌ Ошибка тестирования Whisper: {e}")
        return False

# 🔴 ДОБАВЛЕНО: Проверка всех моделей перед запуском
def check_all_models_before_launch():
    """Проверка всех моделей перед запуском интерфейса"""
    print("\n" + "="*60)
    print("🚀 ПРЕДВАРИТЕЛЬНАЯ ПРОВЕРКА МОДЕЛЕЙ")
    print("="*60)
    
    # 1. Проверка CUDA
    try:
        check_cuda_availability()
        print("✅ CUDA доступен")
    except RuntimeError as e:
        print(f"❌ CUDA не доступен: {e}")
        return False
    
    # 2. Проверка RUNorm
    try:
        test_text = "13,8 миллиардов лет"
        result = normalizer.norm(test_text)
        print(f"✅ RUNorm работает: '{test_text}' -> '{result}'")
    except Exception as e:
        print(f"⚠️ RUNorm не работает: {e}")
    
    # 3. Проверка Silero Stress
    try:
        test_word = "привет"
        result = accentor(test_word)
        print(f"✅ Silero Stress работает: '{test_word}' -> '{result}'")
    except Exception as e:
        print(f"⚠️ Silero Stress не работает: {e}")
    
    # 4. Проверка Whisper
    whisper_ok = test_whisper_model()
    
    if not whisper_ok:
        print("⚠️ ВНИМАНИЕ: Whisper может не работать корректно!")
        print("   Аудио транскрипция будет отключена.")
    
    print("="*60)
    return True
# ================ КОНЕЦ ВСТАВКИ ================

def transcribe_audio(audio_path):
    if not audio_path or asr_processor is None or asr_model is None:
        return ""

    try:
        # 1. Загрузка аудио
        waveform, sample_rate = torchaudio.load(audio_path)

        # 2. Ресемплинг в 16 кГц
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        # 3. Моно
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # 4. В numpy
        input_audio = waveform.squeeze().numpy()

        # 5. Обработка: получаем input_features (без паддинга)
        inputs = asr_processor(
            input_audio,
            sampling_rate=16000,
            return_tensors="pt",
            # ❗ Убираем padding и attention_mask — будем падить вручную
        )

        input_features = inputs.input_features  # [1, 80, T], где T < 3000

        # 🔹 🔥 РУЧНОЙ ПАДДИНГ ДО 3000
        current_length = input_features.shape[-1]
        if current_length < 3000:
            pad_length = 3000 - current_length
            input_features = torch.nn.functional.pad(input_features, (0, pad_length), mode='constant', value=0)
        elif current_length > 3000:
            input_features = input_features[..., :3000]  # обрезаем, если вдруг больше

        print("Input features shape:", input_features.shape)  # Должно быть [1, 80, 3000]

        # 6. Переносим на GPU
        input_features = input_features.to(asr_model.device)

        # 7. Генерация
        with torch.no_grad():
            generated_ids = asr_model.generate(
                input_features=input_features,
                language=None,
                task="transcribe",
            )

        # 8. Децодинг
        transcription = asr_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return transcription.strip().capitalize() + "."

    except Exception as e:
        print("ASR transcription failed:", e)
        return "Ошибка распознавания."
# --- Конец функции ---

def validate_and_transcribe_audio(audio_path):
    """Валидация аудиофайла (без автоматической транскрипции)"""
    if not audio_path:
        return None, "", gr.update(value="", visible=False)
    
    try:
        # 🔴 ИЗМЕНЕНИЕ: Больше НЕ транскрибируем автоматически!
        # Просто возвращаем путь к аудио
        return audio_path, "", gr.update(value="", visible=False)
        
    except Exception as e:
        print("Validation failed:", e)
        return None, "", gr.update(value="⚠️ **Ошибка при проверке аудио**", visible=True)

# 🔴 ДОБАВЛЕНО: Функция для ручной транскрипции
def manual_transcribe_audio(audio_path):
    """Ручная транскрипция аудио"""
    if not audio_path:
        return "", gr.update(value="❌ Нет аудиофайла", visible=True)
    
    if asr_processor is None or asr_model is None:
        return "", gr.update(value="❌ Whisper не загружен", visible=True)
    
    try:
        transcription = transcribe_audio(audio_path)
        if transcription:
            return transcription, gr.update(value="✅ Транскрипция завершена", visible=True)
        else:
            return "", gr.update(value="❌ Не удалось транскрибировать", visible=True)
    except Exception as e:
        print(f"Ошибка ручной транскрипции: {e}")
        return "", gr.update(value=f"❌ Ошибка: {str(e)[:50]}", visible=True)
# ================ КОНЕЦ ЗАМЕНЫ ================

# Check CUDA availability at startup
print("Checking CUDA availability...")
try:
    check_cuda_availability()
except RuntimeError as e:
    print(f"FATAL ERROR: {e}")
    exit(1)

# ================ ИСПРАВЛЕННЫЕ ФУНКЦИИ ДЛЯ ОБРАБОТКИ ТЕКСТА ================

def process_text_with_accent(text):
    """Обработка текста с добавлением ударений через silero-stress с сохранением букв 'ё'"""
    if not text or not text.strip():
        print("❌ Пустой текст для обработки")
        return text

    # 🔴 Если уже есть ударения - возвращаем как есть
    if '+' in text:
        print("✅ Текст уже содержит ударения, пропускаем обработку")
        return text

    try:
        print(f"🎯 Начинаем обработку текста: '{text}'")
        
        # Разбиваем текст на слова и не-слова
        print("🔧 Разбивка на слова...")
        words_and_delimiters = re.split(r'(\W+)', text)
        result_parts = []
        
        print(f"🔍 Найдено {len(words_and_delimiters)} частей")
        
        for i, part in enumerate(words_and_delimiters):
            if part.strip() and re.match(r'^\w+$', part):
                # 🔴 Это слово - обрабатываем через silero-stress
                print(f"🔤 Обработка слова {i+1}: '{part}'")
                try:
                    processed_word = accentor(part)
                    print(f"✅ Слово '{part}' -> '{processed_word}'")
                    
                    # 🔴 🔥 КЛЮЧЕВОЕ ИЗМЕНЕНИЕ: ВОССТАНАВЛИВАЕМ ОРИГИНАЛЬНЫЕ "ё" И "е"
                    # Сохраняем позиции оригинальных букв "ё" и "Ё" в исходном слове
                    original_word = part
                    processed_word_list = list(processed_word)
                    
                    # Для каждой позиции в исходном слове
                    for j, orig_char in enumerate(original_word):
                        if j < len(processed_word_list):
                            # Если в оригинале была "ё" или "Ё", а Silero заменил её на "е" или "Е"
                            if (orig_char in ['ё', 'Ё'] and 
                                processed_word_list[j].lower() == 'е'):
                                # 🔴 Восстанавливаем оригинальную букву "ё" или "Ё"
                                processed_word_list[j] = orig_char
                                print(f"🔄 Восстановлена буква '{orig_char}' на позиции {j}")
                    
                    processed_word = ''.join(processed_word_list)
                    print(f"🔄 После восстановления ё/Ё: '{processed_word}'")
                    
                    # 🔴 ПРОСТОЕ И НАДЕЖНОЕ ВОССТАНОВЛЕНИЕ РЕГИСТРА
                    if part and processed_word:
                        # Сохраняем регистр первой буквы
                        if part[0].isupper():
                            processed_word = processed_word[0].upper() + processed_word[1:]
                            print(f"🔠 Восстановлен верхний регистр: '{processed_word}'")
                        else:
                            processed_word = processed_word[0].lower() + processed_word[1:]
                            print(f"🔡 Восстановлен нижний регистр: '{processed_word}'")
                        
                        result_parts.append(processed_word)
                    else:
                        print(f"⚠️ Пропускаем слово '{part}'")
                        result_parts.append(part)
                except Exception as e:
                    print(f"❌ Silero stress failed for word '{part}': {e}")
                    result_parts.append(part)
            else:
                # 🔴 Не слово (пунктуация, пробелы и т.д.) - оставляем как есть
                if part.strip():
                    print(f"📎 Не-слово: '{part}'")
                result_parts.append(part)
        
        final_result = ''.join(result_parts)
        print(f"🎯 Финальный текст с ударениями: '{final_result}'")
        print(f"📊 Сравнение: Исходный: '{text}' -> Обработанный: '{final_result}'")
        return final_result
        
    except Exception as e:
        print(f"❌ Text processing failed: {e}")
        print(f"🔍 Traceback: {traceback.format_exc()}")
        return text

def normalize_gen_text(gen_text):
    """Нормализация только текста для генерации (silero-stress)"""
    # Блокировка обработки путей к файлам
    if gen_text and isinstance(gen_text, str):
        if any(bad in gen_text for bad in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac", "/home/", "/root/"]):
            gen_text = ""
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ ТЕКСТА ДЛЯ ГЕНЕРАЦИИ")
    print(f"📖 Текст до обработки: '{gen_text}'")
    
    # 🔴 ПРОВЕРКА: Содержит ли текст цифры?
    contains_digits = any(char.isdigit() for char in gen_text) if gen_text else False
    
    # Шаг 1: Нормализация через RUNorm (только если есть цифры)
    if gen_text and contains_digits:
        print("🔢 Текст содержит цифры, запускаем RUNorm...")
        print("🔧 Шаг 1: Нормализация через RUNorm...")
        try:
            normalized_text = normalizer.norm(gen_text)
            print(f"📝 После RUNorm: '{normalized_text}'")
        except Exception as norm_error:
            print(f"❌ Ошибка RUNorm: {norm_error}")
            normalized_text = gen_text
    else:
        if gen_text and not contains_digits:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
        normalized_text = gen_text if gen_text else ""
    
    # Шаг 2: Обработка ударений через silero-stress
    if normalized_text:
        print("🔧 Шаг 2: Обработка ударений...")
        processed_gen = process_text_with_accent(normalized_text)
        print(f"✅ Текст после обработки: '{processed_gen}'")
    else:
        processed_gen = ""
    
    print("=" * 60)
    
    return processed_gen

def normalize_ref_text(ref_text):
    """Нормализация референсного текста (RUNorm + silero-stress)"""
    # Блокировка обработки путей к файлам
    if ref_text and isinstance(ref_text, str):
        if any(bad in ref_text for bad in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac", "/home/", "/root/"]):
            ref_text = ""
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ РЕФЕРЕНСНОГО ТЕКСТА")
    print(f"📖 Референс текст до обработки: '{ref_text}'")
    
    # 🔴 ПРОВЕРКА: Содержит ли текст цифры?
    contains_digits = any(char.isdigit() for char in ref_text) if ref_text else False
    
    # Шаг 1: Нормализация через RUNorm (только если есть цифры)
    if ref_text and contains_digits:
        print("🔢 Текст содержит цифры, запускаем RUNorm...")
        print("🔧 Шаг 1: Нормализация через RUNorm...")
        try:
            normalized_text = normalizer.norm(ref_text)
            print(f"📝 После RUNorm: '{normalized_text}'")
        except Exception as norm_error:
            print(f"❌ Ошибка RUNorm: {norm_error}")
            normalized_text = ref_text
    else:
        if ref_text and not contains_digits:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
        normalized_text = ref_text if ref_text else ""
    
    # Шаг 2: Обработка ударений через silero-stress
    if normalized_text:
        print("🔧 Шаг 2: Обработка ударений...")
        processed_ref = process_text_with_accent(normalized_text)
        print(f"✅ Референс текст после обработки: '{processed_ref}'")
    else:
        processed_ref = ""
    
    print("=" * 60)
    return processed_ref

# ================ НОВЫЕ ФУНКЦИИ НОРМАЛИЗАЦИИ С РАЗДЕЛЬНЫМИ ШАГАМИ ================
def normalize_ref_text_with_steps(ref_text):
    """Нормализация референсного текста с раздельными шагами RUNorm → Silero Stress"""
    if not ref_text or not ref_text.strip():
        return ref_text, "❌ Пустой текст", "❌ Пустой текст"
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ РЕФЕРЕНСНОГО ТЕКСТА (2 ШАГА)")
    print(f"📖 Исходный текст: '{ref_text}'")
    
    step1_result = ""
    step2_result = ""
    step1_status = ""
    
    try:
        # 🔴 ПРОВЕРКА: Содержит ли текст цифры?
        contains_digits = any(char.isdigit() for char in ref_text)
        
        if contains_digits:
            print("🔢 Текст содержит цифры, запускаем RUNorm...")
            # 🔴 ШАГ 1: Нормализация через RUNorm (только если есть цифры)
            print("🔧 Шаг 1: Запуск RUNorm...")
            step1_result = normalizer.norm(ref_text)
            print(f"✅ RUNorm результат: '{step1_result}'")
            step1_status = f"✅ RUNorm (цифры найдены): '{step1_result}'"
        else:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
            step1_result = ref_text  # Пропускаем RUNorm
            step1_status = "⏭️ RUNorm пропущен (нет цифр)"
        
        # 🔴 ШАГ 2: Обработка ударений через Silero Stress
        print("🔧 Шаг 2: Запуск Silero Stress...")
        step2_result = process_text_with_accent(step1_result)
        print(f"✅ Silero Stress результат: '{step2_result}'")
        
        print("=" * 60)
        print(f"🎯 Итог: '{ref_text}' → '{step2_result}'")
        print(f"📏 Статистика:")
        print(f"   Исходный текст: {len(ref_text)} символов")
        print(f"   После обработки: {len(step2_result)} символов")
        
        # Возвращаем обработанный текст и промежуточные результаты
        return step2_result, step1_status, f"✅ Silero Stress: '{step2_result}'"
        
    except Exception as e:
        print(f"❌ Ошибка нормализации: {e}")
        traceback.print_exc()
        return ref_text, f"❌ Ошибка: {str(e)[:50]}", "❌ Ошибка на шаге 2"

def normalize_gen_text_with_steps(gen_text):
    """Нормализация текста генерации с раздельными шагами RUNorm → Silero Stress"""
    if not gen_text or not gen_text.strip():
        return gen_text, "❌ Пустой текст", "❌ Пустой текст"
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ ТЕКСТА ГЕНЕРАЦИИ (2 ШАГА)")
    print(f"📖 Исходный текст: '{gen_text}'")
    
    step1_result = ""
    step2_result = ""
    step1_status = ""
    
    try:
        # 🔴 ПРОВЕРКА: Содержит ли текст цифры?
        contains_digits = any(char.isdigit() for char in gen_text)
        
        if contains_digits:
            print("🔢 Текст содержит цифры, запускаем RUNorm...")
            # 🔴 ШАГ 1: Нормализация через RUNorm (только если есть цифры)
            print("🔧 Шаг 1: Запуск RUNorm...")
            step1_result = normalizer.norm(gen_text)
            print(f"✅ RUNorm результат: '{step1_result}'")
            step1_status = f"✅ RUNorm (цифры найдены): '{step1_result}'"
        else:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
            step1_result = gen_text  # Пропускаем RUNorm
            step1_status = "⏭️ RUNorm пропущен (нет цифр)"
        
        # 🔴 ШАГ 2: Обработка ударений через Silero Stress
        print("🔧 Шаг 2: Запуск Silero Stress...")
        step2_result = process_text_with_accent(step1_result)
        print(f"✅ Silero Stress результат: '{step2_result}'")
        
        print("=" * 60)
        print(f"🎯 Итог: '{gen_text}' → '{step2_result}'")
        
        # Возвращаем обработанный текст и промежуточные результаты
        return step2_result, step1_status, f"✅ {step2_result}"
        
    except Exception as e:
        print(f"❌ Ошибка нормализации: {e}")
        traceback.print_exc()
        return gen_text, f"❌ Ошибка: {str(e)[:50]}", "❌ Ошибка на шаге 2"

def get_current_seed_display():
    """Получить текущее значение seed для отображения"""
    global last_seed
    # Если last_seed еще не установлен, генерируем случайный
    if last_seed == -1:
        last_seed = np.random.randint(0, 2**31 - 1)
        print(f"🎲 Инициализирован начальный seed: {last_seed}")
    return last_seed

# 🔴 ИСПРАВЛЕННАЯ ФУНКЦИЯ: Обновление статуса загрузки модели
def update_model_loading_status(model_name):
    """Обновление статуса загрузки модели"""
    global loaded_models
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    # Объединяем конфиги моделей
    all_models_config = get_all_models_config()
    
    if model_name not in all_models_config:
        return gr.update(value=f"<div class='model-status model-error'>❌ Модель {model_name} не найдена</div>", visible=True)
    
    # 🔴 ИСПРАВЛЕНИЕ: Проверяем, загружена ли модель в данный момент
    if model_name in loaded_models:
        model_type = "🔧 Локальная" if all_models_config[model_name]['repo'] == "local" else "🌐 HuggingFace"
        return gr.update(value=f"<div class='model-status model-loaded'>✅ {model_name} ({model_type}) загружена</div>", visible=True)
    else:
        # 🔴 ИСПРАВЛЕНИЕ: Показываем статус "не загружена" только если модель не загружена
        model_type = "🔧 Локальная" if all_models_config[model_name]['repo'] == "local" else "🌐 HuggingFace"
        return gr.update(value=f"<div class='model-status model-loading'>🔄 {model_name} ({model_type}) не загружена</div>", visible=True)

# 🔴 ИСПРАВЛЕННАЯ ФУНКЦИЯ: Загрузка модели с обновлением статуса
def load_model_with_status(model_name, progress=gr.Progress()):
    """Загрузка модели с обновлением статуса"""
    global loaded_models
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    # 🔴 ПРОВЕРКА ОСТАНОВКИ В НАЧАЛЕ
    try:
        check_stop_generation()
    except KeyboardInterrupt:
        print("🛑 Загрузка модели остановлена пользователем")
        return gr.update(value="<div class='model-status model-error'>🛑 Загрузка остановлена</div>", visible=True)
    
    # Если модель уже загружена, просто возвращаем статус
    if model_name in loaded_models:
        return update_model_loading_status(model_name)
    
    try:
        # 🔴 ПРОВЕРКА ОСТАНОВКИ ПЕРЕД НАЧАЛОМ ЗАГРУЗКИ
        check_stop_generation()
        
        # Показываем статус загрузки
        progress(0.1, desc="🔄 Загрузка модели...")
        
        # 🔴 ОБНОВЛЕННЫЙ CALLBACK С ПРОВЕРКОЙ ОСТАНОВКИ
        def progress_callback_with_stop(msg, p):
            # Проверяем остановку в каждом шаге прогресса
            try:
                check_stop_generation()
            except KeyboardInterrupt:
                print(f"🛑 Остановка на этапе: {msg}")
                raise
            
            # Обновляем прогресс
            progress(p, desc=msg)
        
        # Загружаем модель с callback, который проверяет остановку
        model = load_model_with_progress(model_name, progress_callback_with_stop)
        
        if model:
            # Успешная загрузка
            return update_model_loading_status(model_name)
        else:
            return gr.update(value="<div class='model-status model-error'>❌ Загрузка отменена</div>", visible=True)
            
    except KeyboardInterrupt:
        print("🛑 Загрузка модели прервана пользователем")
        return gr.update(value="<div class='model-status model-error'>🛑 Загрузка прервана пользователем</div>", visible=True)
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки: {str(e)}"
        print(f"Model loading error: {traceback.format_exc()}")
        return gr.update(value=f"<div class='model-status model-error'>{error_msg}</div>", visible=True)

def on_model_select(model_name):
    """Обработчик выбора модели в dropdown"""
    global MODELS_CONFIG, local_models_config
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    # Объединяем конфиги моделей
    all_models_config = {**MODELS_CONFIG, **local_models_config}
    
    if model_name not in all_models_config:
        return gr.update(value=f"<div class='model-status model-error'>❌ Модель {model_name} не найдена</div>", visible=True)
    
    # 🔴 ИСПРАВЛЕНИЕ: Просто показываем текущий статус без автоматической загрузки
    return update_model_loading_status(model_name)

def synthesize(
    ref_audio,
    ref_text,
    gen_text,
    remove_silence,
    seed_input_value,
    remember_seed_checkbox,
    model_choice,
    cross_fade_duration=0.15,
    nfe_step=32,
    speed=1.0,
    sway_sampling_coef=-1,
    cfg_strength=2,
    audio_format="wav",
    bitrate="192k",
    progress=gr.Progress()
):
    global stop_generation, last_seed, remember_seed, generation_thread
    
    # 🔴 ОБНОВЛЯЕМ ФЛАГ ЗАПОМИНАНИЯ СИДА
    remember_seed = remember_seed_checkbox
    
    # 🔴 СБРАСЫВАЕМ ВСЕ ФЛАГИ ГЕНЕРАЦИИ (ЗАМЕНА reset_stop_flag)
    reset_generation_flags()
    
    # 🔴 ПРОВЕРЯЕМ НЕ БЫЛА ЛИ УЖЕ ОСТАНОВКА
    try:
        check_stop_generation()
    except KeyboardInterrupt:
        gr.Warning("🛑 Генерация остановлена")
        current_model_status = update_model_loading_status(model_choice)  # ← ДОБАВЬТЕ ЭТУ СТРОКУ
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    # 🔴 СРАЗУ ОБНОВЛЯЕМ СТАТУС МОДЕЛИ
    current_model_status = update_model_loading_status(model_choice)
    
    # 🔴 РАННЯЯ ПРОВЕРКА STOP_FLAG (это уже есть в вашем коде, оставьте)
    if stop_generation:
        gr.Warning("🛑 Генерация остановлена")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    # 🔄 ОБНОВЛЯЕМ СТАТУС ЗАГРУЗКИ
    progress(0.05, desc="🔄 Подготовка...")
    
    # 🎯 ПРОВЕРКА ВХОДНЫХ ДАННЫХ
    if not ref_audio:
        gr.Warning("❌ Пожалуйста, предоставьте референсное аудио")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    if not gen_text or not gen_text.strip():
        gr.Warning("❌ Пожалуйста, введите текст для генерации")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
        
    if not ref_text or not ref_text.strip():
        gr.Warning("❌ Пожалуйста, предоставьте референсный текст")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    # 🔐 ЗАЩИТА ОТ ПУТЕЙ К ФАЙЛАМ
    if gen_text and isinstance(gen_text, str):
        if any(x in gen_text for x in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac"]):
            gr.Warning("⚠️ Поле 'Text to Generate' содержит путь к файлу. Пожалуйста, введите текст вручную.")
            return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    # 🔴 ПРОВЕРКА STOP_FLAG
    if stop_generation:
        gr.Warning("🛑 Генерация остановлена")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    # 🎲 УСТАНОВКА SEED - КЛЮЧЕВОЕ ИЗМЕНЕНИЕ!
    current_seed = seed_input_value
    if seed_input_value is None or seed_input_value < 0 or seed_input_value > 2**31 - 1:
        # 🔴 ГЕНЕРИРУЕМ СЛУЧАЙНЫЙ СИД
        current_seed = np.random.randint(0, 2**31 - 1)
        print(f"🎲 Сгенерирован случайный seed: {current_seed}")
    
    # 🔴 ЗАПОМИНАЕМ СИД ЕСЛИ ВКЛЮЧЕНА ГАЛОЧКА
    if remember_seed:
        last_seed = current_seed
        print(f"💾 Seed сохранен: {current_seed}")
    
    torch.manual_seed(int(current_seed))
    
    # 🔴 ВЫВОДИМ ИНФОРМАЦИЮ О СИДЕ
    print(f"🎲 Используется seed: {current_seed} (запомнить: {remember_seed})")

    # 📝 ПОДГОТОВКА ТЕКСТА (БЕЗ АВТОМАТИЧЕСКОЙ НОРМАЛИЗАЦИИ)
    progress(0.08, desc="📝 Подготовка текста...")

    # Используем текст как есть (пользователь должен нормализовать вручную через кнопку)
    processed_ref_text = ref_text.strip() if ref_text else ""
    processed_gen_text = gen_text.strip() if gen_text else ""

    # --- ✅ Инициализация processed_ref_text_final ДО try ---
    processed_ref_text_final = processed_ref_text
    
    print(f"🎯 Референс текст (как есть): '{processed_ref_text}'")
    print(f"🎯 Генерационный текст (как есть): '{processed_gen_text}'")  
    print("ℹ️ Автоматическая нормализация отключена. Используйте кнопки нормализации перед генерацией.")

    try:
        # --- 1. ОТЛОЖЕННАЯ ЗАГРУЗКА МОДЕЛЕЙ ---
        progress(0.1, desc="🔍 Проверка моделей...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # 📥 ЗАГРУЗКА ОСНОВНОЙ МОДЕЛИ
        progress(0.15, desc="📥 Загрузка модели TTS...")
        model = ensure_model(model_choice, lambda msg, p: progress(p, desc=msg))
        
        # 🔴 ОБНОВЛЯЕМ СТАТУС ПОСЛЕ ЗАГРУЗКИ МОДЕЛИ
        if model:
            current_model_status = update_model_loading_status(model_choice)
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # 🔊 ЗАГРУЗКА ВОКОДЕРА (ОТЛОЖЕННАЯ)
        progress(0.2, desc="🔊 Загрузка вокодера...")
        vocoder = ensure_vocoder()
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
            
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки моделей: {str(e)}"
        gr.Warning(error_msg)
        print(f"Model loading error: {traceback.format_exc()}")
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    # 🎯 ПРОВЕРКА УСПЕШНОЙ ЗАГРУЗКИ
    if model is None or vocoder is None:
        gr.Warning("❌ Не удалось загрузить модели")
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    device = torch.device("cuda")

    try:
        # --- 2. ПРЕДОБРАБОТКА АУДИО И ТЕКСТА ---
        progress(0.3, desc="🔧 Предобработка аудио...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        try:
            ref_audio_proc, processed_ref_text_final = preprocess_ref_audio_text(
                ref_audio,
                processed_ref_text,  # Используем текст как есть
                show_info=gr.Info
            )
        except Exception as e:
            error_msg = f"❌ Ошибка предобработки: {str(e)}"
            gr.Warning(error_msg)
            traceback.print_exc()
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # --- 3. ГЕНЕРАЦИЯ АУДИО ---
        progress(0.5, desc="🎵 Генерация аудио...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        try:
            final_wave, final_sample_rate, combined_spectrogram = infer_process(
                ref_audio_proc,
                processed_ref_text_final,
                processed_gen_text,  # Используем текст как есть
                model,
                vocoder,
                cross_fade_duration=cross_fade_duration,
                nfe_step=nfe_step,
                speed=speed,
                sway_sampling_coef=sway_sampling_coef,
                cfg_strength=cfg_strength,
                show_info=gr.Info,
                progress=progress,
            )
        except Exception as e:
            error_msg = f"❌ Ошибка генерации аудио: {str(e)}"
            gr.Warning(error_msg)
            traceback.print_exc()
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # --- 4. УДАЛЕНИЕ ТИШИНЫ (ОПЦИОНАЛЬНО) ---
        if remove_silence:
            progress(0.7, desc="🔇 Удаление тишины...")
            
            # 🔴 ПРОВЕРКА STOP_FLAG
            if stop_generation:
                gr.Warning("🛑 Генерация остановлена")
                return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
            
            try:
                with tempfile.NamedTemporaryFile(suffix=".wav", **tempfile_kwargs) as f:
                    temp_path = f.name
                    sf.write(temp_path, final_wave, final_sample_rate)
                    remove_silence_for_generated_wav(temp_path)
                    final_wave_tensor, _ = torchaudio.load(temp_path)
                    final_wave = final_wave_tensor.squeeze().cpu().numpy()
            except Exception as e:
                print("⚠️ Удаление тишины не удалось:", e)
                # Не прерываем выполнение, продолжаем с оригинальным аудио

        # --- 5. ЭКСПОРТ В ФОРМАТ ---
        progress(0.8, desc="💾 Сохранение результата...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # 🎵 ПОДГОТОВКА АУДИО ДЛЯ ЭКСПОРТА
        safe_kwargs = {k: v for k, v in tempfile_kwargs.items() if k != 'delete'}

        with tempfile.NamedTemporaryFile(suffix=f".{audio_format}", delete=False, **safe_kwargs) as tmp:
            temp_output_path = tmp.name

        wave = final_wave
        if wave.ndim == 1:
            channels = 1
        else:
            channels = min(wave.shape[0], 2)
            wave = wave[:channels].T

        # 🔧 НОРМАЛИЗАЦИЯ АУДИО
        if np.max(np.abs(wave)) == 0:
            wave_int16 = np.zeros(wave.shape, dtype=np.int16)
        else:
            wave_int16 = np.int16(wave / np.max(np.abs(wave)) * 32767)

        # 💾 СОХРАНЕНИЕ В ВЫБРАННОМ ФОРМАТЕ
        audio_segment = AudioSegment(
            wave_int16.tobytes(),
            frame_rate=final_sample_rate,
            sample_width=2,
            channels=channels
        )

        try:
            if audio_format == "mp3":
                audio_segment.export(temp_output_path, format="mp3", bitrate=bitrate)
            elif audio_format == "ogg":
                audio_segment.export(temp_output_path, format="ogg", bitrate=bitrate)
            elif audio_format == "flac":
                audio_segment.export(temp_output_path, format="flac")
            else:
                audio_segment.export(temp_output_path, format="wav")
        except Exception as e:
            error_msg = f"❌ Ошибка экспорта аудио: {str(e)}"
            gr.Warning(error_msg)
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # 📁 СОЗДАНИЕ ФИНАЛЬНОГО ФАЙЛА
        timestamp = int(time.time())
        safe_text = f"audio_{timestamp}"
        final_output_path = os.path.join(os.path.dirname(temp_output_path), f"{safe_text}.{audio_format}")

        try:
            if os.path.exists(final_output_path):
                os.remove(final_output_path)
            os.rename(temp_output_path, final_output_path)
        except Exception as e:
            error_msg = f"❌ Ошибка создания файла: {str(e)}"
            gr.Warning(error_msg)
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # --- 6. СОХРАНЕНИЕ СПЕКТРОГРАММЫ ---
        progress(0.9, desc="📊 Сохранение спектрограммы...")

        spectrogram_path = None
        try:
            # 🔴 ИСПРАВЛЕНИЕ: Используем delete=False для временных файлов
            with tempfile.NamedTemporaryFile(suffix=".png", delete=False, **tempfile_kwargs) as tmp_spectrogram:
                spectrogram_path = tmp_spectrogram.name
            
            # 🔴 Явно сохраняем спектрограмму
            save_spectrogram(combined_spectrogram, spectrogram_path)
            print(f"✅ Спектрограмма сохранена: {spectrogram_path}")
            
        except Exception as e:
            print(f"⚠️ Сохранение спектрограммы не удалось: {e}")
            spectrogram_path = None

        progress(1.0, desc="✅ Готово!")

        # 🔴 ДЕБАГ ИНФОРМАЦИЯ
        print("=" * 50)
        print("🎯 ДЕБАГ ИНФОРМАЦИЯ О ФАЙЛАХ:")
        print(f"🎵 Аудио файл: {final_output_path}")
        print(f"🎵 Аудио существует: {os.path.exists(final_output_path)}")

        print(f"📊 Спектрограмма: {spectrogram_path}")
        if spectrogram_path:
            exists = os.path.exists(spectrogram_path)
            print(f"📊 Спектрограмма существует: {exists}")
            if exists:
                size = os.path.getsize(spectrogram_path)
                print(f"📊 Размер спектрограммы: {size} bytes")
        else:
            print("📊 Спектрограмма: None")
        print("=" * 50)

        # 🎉 ВОЗВРАТ УСПЕШНОГО РЕЗУЛЬТАТА
        return (
            final_output_path,
            spectrogram_path,
            processed_ref_text_final,  # Возвращаем текст как есть
            processed_gen_text,        # Возвращаем текст как есть
            gr.update(value=current_seed),
            current_model_status
        )

    except Exception as e:
        error_msg = f"❌ Неожиданная ошибка: {str(e)}"
        print(f"Unexpected error in synthesize: {traceback.format_exc()}")
        gr.Warning(error_msg)
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    finally:
        # 🧹 ОЧИСТКА ПАМЯТИ (ЕСЛИ НЕ БЫЛО ОСТАНОВКИ)
        if not stop_generation:
            try:
                torch.cuda.empty_cache()
                gc.collect()
            except Exception as e:
                print("⚠️ Ошибка при очистке памяти:", e)
                
# 🔴 ДОБАВЛЕНО: Функция обновления статуса моделей
def update_model_check_status():
    """Обновить статус проверки моделей"""
    try:
        # Проверяем все модели
        whisper_ok = test_whisper_model() if asr_processor else False
        
        # Проверяем RUNorm
        runorm_ok = False
        try:
            test_text = "13,8 миллиардов лет"
            result = normalizer.norm(test_text)
            runorm_ok = True
        except:
            runorm_ok = False
        
        # Проверяем Silero Stress
        silero_ok = False
        try:
            test_word = "привет"
            result = accentor(test_word)
            silero_ok = True
        except:
            silero_ok = False
        
        # Проверяем CUDA
        cuda_ok = torch.cuda.is_available()
        
        # Проверяем авторизацию HuggingFace
        hf_auth_ok = HF_TOKEN is not None
        
        # Формируем HTML статус
        html = """
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px;">
            <h3>📊 Статус моделей:</h3>
            <ul style="list-style: none; padding: 0;">
                <li>🎯 <strong>CUDA:</strong> {}</li>
                <li>🔐 <strong>HuggingFace Auth:</strong> {}</li>
                <li>🤖 <strong>Whisper ASR:</strong> {}</li>
                <li>📝 <strong>RUNorm:</strong> {}</li>
                <li>🔊 <strong>Silero Stress:</strong> {}</li>
            </ul>
        </div>
        """.format(
            "✅ Доступен" if cuda_ok else "❌ Недоступен",
            "✅ Авторизован" if hf_auth_ok else "❌ Нет токена",
            "✅ Готов" if whisper_ok else "❌ Не загружен",
            "✅ Готов" if runorm_ok else "⚠️ Частично",
            "✅ Готов" if silero_ok else "⚠️ Частично"
        )
        
        return gr.update(value=html)
    except Exception as e:
        return gr.update(value=f"<div class='model-status model-error'>❌ Ошибка проверки: {str(e)[:100]}</div>")

# 🔴 ДОБАВЛЕНО: Функция-обертка для обновления токена
def update_token_handler(new_token):
    return update_hf_token(new_token)

with gr.Blocks(title="ESpeech-TTS", css="""
    .error-markdown {
        color: red !important;
        font-weight: bold;
        text-align: center;
        font-size: 18px !important;
        margin: 10px 0;
        animation: fadeIn 0.3s, fadeOut 0.3s 4.7s forwards;
    }
    .model-status {
        padding: 10px;
        border-radius: 5px;
        margin: 10px 0;
        text-align: center;
        font-weight: bold;
    }
               
    /* 🔴 УВЕЛИЧЕННЫЕ СТИЛИ ДЛЯ КНОПОК НОРМАЛИЗАЦИИ */
    .norm-button {
        font-size: 16px !important;  /* ⬅️ Увеличиваем шрифт */
        padding: 12px 24px !important;  /* ⬅️ Увеличиваем отступы */
        background: linear-gradient(45deg, #007bff, #0056b3) !important;
        color: white !important;
        font-weight: bold !important;
        margin: 10px 0 !important;  /* ⬅️ Увеличили вертикальные отступы */
        border-radius: 8px !important;  /* ⬅️ Скругленные углы */
    }
    .norm-button:hover {
        background: linear-gradient(45deg, #0056b3, #003d82) !important;
        transform: translateY(-2px) !important;
        box-shadow: 0 4px 8px rgba(0, 86, 179, 0.3) !important;
    }
    
    /* 🔴 СТИЛИ ДЛЯ ПОЛЕЙ ПРОГРЕССА */
    .step-result-box {
        background: #f8f9fa !important;
        border-left: 4px solid #28a745 !important;
        border-radius: 5px !important;
        padding: 10px !important;
        margin: 5px 0 !important;
    }
    .step-label {
        font-weight: bold;
        color: #155724;
    }
    .step-text {
        color: #0c5460;
        font-style: italic;
        font-size: 14px;
        word-break: break-word;
    }
    
    /* 🔴 СТИЛИ ДЛЯ КНОПКИ ОСТАНОВКИ */
    .stop-button {
        background: linear-gradient(45deg, #dc3545, #c82333) !important;
        color: white !important;
        font-weight: bold !important;
        border: none !important;
        box-shadow: 0 4px 6px rgba(220, 53, 69, 0.3) !important;
    }
    .stop-button:hover {
        background: linear-gradient(45deg, #c82333, #a71d2a) !important;
        transform: translateY(-2px) !important;
        box-shadow: 0 6px 8px rgba(220, 53, 69, 0.4) !important;
    }               
    .model-loaded {
        background: #d4edda;
        color: #155724;
        border: 1px solid #c3e6cb;
    }
    .model-error {
        background: #f8d7da;
        color: #721c24;
        border: 1px solid #f5c6cb;
    }
    .model-loading {
        background: #fff3cd;
        color: #856404;
        border: 1px solid #ffeaa7;
    }
    .loading-status {
        padding: 10px;
        margin: 10px 0;
        border-radius: 5px;
        text-align: center;
    }
    .local-model-input {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        border: 1px solid #dee2e6;
        margin: 10px 0;
    }
    @keyframes fadeIn { from { opacity: 0; } to { opacity: 1; } }
    @keyframes fadeOut { from { opacity: 1; } to { opacity: 0; } }
    """) as app:
    gr.Markdown("# ESpeech-TTS")

    # 🔴🔴🔴 БЛОК СТАТУСА МОДЕЛЕЙ 🔴🔴🔴
    with gr.Accordion("📊 Статус моделей", open=False):
        model_check_status = gr.HTML(
            value="<div class='model-status model-loading'>🔄 Проверка моделей...</div>"
        )
        check_models_btn = gr.Button("🔄 Проверить модели", variant="secondary")

    # 🔴 ИНФОРМАЦИЯ О АВТОРИЗАЦИИ
    auth_status_text = "✅ Авторизация HuggingFace выполнена!" if HF_TOKEN else "⚠️ Авторизация HuggingFace не выполнена. Закрытые репозитории недоступны."
    gr.Markdown(f"**🔐 Статус авторизации:** {auth_status_text}")

    gr.Markdown("💡 **Совет:** Добавьте символ '+' для ударения (например, 'прив+ет')")
    gr.Markdown("🎲 **Совет:** Seed отвечает за вариативность генерации. Вы можете подбирать для себя удачное значение или каждый раз использовать случайный seed")
    gr.Markdown("🚀 **CUDA Required:** This application requires GPU with CUDA support")

    # 🔴 НОВЫЙ БЛОК: Добавление локальных моделей
    with gr.Accordion("🔧 Добавить локальную модель", open=False):
        # 🔴 БЛОК ДЛЯ ТОКЕНА
        gr.Markdown("### 🔐 Настройка токена HuggingFace")
        gr.Markdown("Токен требуется для доступа к закрытым репозиториям на HuggingFace")
        
        with gr.Row():
            hf_token_input = gr.Textbox(
                label="HuggingFace Token",
                placeholder="hf_xxxxxxxxxxxxxxxxxxxx",
                type="password",  # ⚠️ Скрывает ввод
                scale=4
            )
            update_token_btn = gr.Button("🔐 Обновить токен", variant="secondary", scale=1)
            token_status = gr.HTML(visible=False)
        
        gr.Markdown("### 📁 Добавление локальной модели")
        with gr.Row():
            local_model_path = gr.Textbox(
                label="Путь к файлу модели",
                placeholder="hf://SWivid/F5-TTS/F5TTS_v1_Base/model_1250000.safetensors или /path/to/model.safetensors",
                scale=3
            )
            local_vocab_path = gr.Textbox(
                label="Путь к файлу словаря", 
                placeholder="hf://SWivid/F5-TTS/F5TTS_v1_Base/vocab.txt или /path/to/vocab.txt",
                scale=3
            )
            local_model_name = gr.Textbox(
                label="Имя модели",
                placeholder="My Local Model",
                scale=2
            )
        with gr.Row():
            add_local_model_btn = gr.Button("➕ Добавить локальную модель", variant="secondary")
            local_model_status = gr.HTML(visible=False)

    # --- 🔴 ИСПРАВЛЕННЫЙ СТАТУС МОДЕЛИ ---
    all_models = get_all_models_config()
    initial_model = list(all_models.keys())[0] if all_models else "ESpeech-TTS-1_SFT-95K"
    initial_status = update_model_loading_status(initial_model)

    model_status = gr.HTML(
        value=initial_status["value"],
        visible=initial_status["visible"]
    )

    with gr.Row():
        with gr.Column():
            # --- Выбор модели ---
            model_choice = gr.Dropdown(
                choices=list(all_models.keys()),
                value=initial_model,
                label="Выберите модель",
                info="🌐 HuggingFace модели или 🔧 локальные модели"
            )
            
            # --- 🔴 ИСПРАВЛЕННАЯ КНОПКА ЗАГРУЗКИ МОДЕЛИ ---
            load_model_btn = gr.Button("🔄 Загрузить выбранную модель", variant="secondary")
            clear_models_btn = gr.Button("🗑️ Очистить загруженные модели", variant="stop")
            
        with gr.Column():
            ref_audio_input = gr.Audio(label="Reference Audio", type="filepath")

            with gr.Row():
                check_audio_btn = gr.Button("🔍 Проверить аудио", variant="secondary")
                transcribe_btn = gr.Button("🎤 Транскрибировать аудио", variant="secondary")
            
            audio_check_status = gr.HTML(visible=False)
            
            ref_text_input = gr.Textbox(
                label="Reference Text",
                lines=2,
                placeholder="Введите текст вручную или используйте кнопку транскрипции"
            )
            
            # 🔴 КНОПКА НОРМАЛИЗАЦИИ ДЛЯ РЕФЕРЕНСНОГО ТЕКСТА (ПОД ИНПУТОМ)
            normalize_ref_btn = gr.Button("📝 Нормализовать референс (RUNorm → Silero)", 
                                        variant="secondary", 
                                        size="lg",  # ⬅️ Увеличен размер
                                        elem_classes="norm-button")

        with gr.Column():
            gen_text_input = gr.Textbox(
                label="Text to Generate",
                lines=5,
                max_lines=20,
                placeholder="Enter text to synthesize..."
            )
    
            # 🔴 КНОПКА НОРМАЛИЗАЦИИ ДЛЯ ТЕКСТА ГЕНЕРАЦИИ (ПОД ИНПУТОМ)
            normalize_gen_btn = gr.Button("✏️ Нормализовать текст (RUNorm → Silero)", 
                                        variant="secondary", 
                                        size="lg",  # ⬅️ Увеличен размер
                                        elem_classes="norm-button")

    # 🔴 Поля для отображения прогресса нормализации референса
    with gr.Accordion("📊 Прогресс нормализации референса", open=False):
        ref_norm_step1 = gr.Textbox(label="Шаг 1: RUNorm результат", interactive=False, visible=True)
        ref_norm_step2 = gr.Textbox(label="Шаг 2: Silero Stress результат", interactive=False, visible=True)

    # 🔴 Поля для отображения прогресса нормализации текста генерации
    with gr.Accordion("📊 Прогресс нормализации текста", open=False):
        gen_norm_step1 = gr.Textbox(label="Шаг 1: RUNorm результат", interactive=False, visible=True)
        gen_norm_step2 = gr.Textbox(label="Шаг 2: Silero Stress результат", interactive=False, visible=True)


    # --- 🔴 ИСПРАВЛЕННЫЕ ОБРАБОТЧИКИ ДЛЯ ЗАГРУЗКИ МОДЕЛИ ---
    def on_model_select(model_name):
        """Обработчик выбора модели в dropdown"""
        if model_name:
            return update_model_loading_status(model_name)
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)

    # ================ ОБРАБОТЧИКИ КНОПОК ================
    
    # 1. Обработчик для проверки аудио
    check_audio_btn.click(
        fn=check_audio_file,
        inputs=[ref_audio_input],
        outputs=[audio_check_status]
    )
    
    # 2. Обработчик для проверки моделей
    check_models_btn.click(
        fn=update_model_check_status,
        outputs=[model_check_status]
    )

    # 3. Обработчик для обновления токена
    update_token_btn.click(
        fn=update_token_handler,
        inputs=hf_token_input,
        outputs=token_status
    )

    # 4. Обработчик для добавления локальной модели
    add_local_model_btn.click(
        fn=add_local_model_with_auth,  # ⬅️ НОВАЯ ФУНКЦИЯ С АВТОРИЗАЦИЕЙ
        inputs=[local_model_path, local_vocab_path, local_model_name, hf_token_input],
        outputs=[model_choice, local_model_status]
    )

    # 5. Обработчик для очистки моделей
    clear_models_btn.click(
        fn=clear_loaded_models,
        outputs=[model_status, local_model_path, local_vocab_path, local_model_name, model_choice]
    )

    # 6. Автозагрузка модели при выборе - ТОЛЬКО СТАТУС
    model_choice.change(
        fn=on_model_select,
        inputs=model_choice,
        outputs=model_status
    )

    # 7. Кнопка принудительной загрузки
    load_model_btn.click(
        fn=load_model_with_status,
        inputs=[model_choice],
        outputs=model_status
    )

    # 8. Кнопка транскрипции
    transcribe_btn.click(
        fn=manual_transcribe_audio,
        inputs=[ref_audio_input],
        outputs=[ref_text_input, audio_check_status]  # Используем существующий статус
    )

    # 🔴 ОБРАБОТЧИК ДЛЯ КНОПКИ НОРМАЛИЗАЦИИ РЕФЕРЕНСА
    normalize_ref_btn.click(
        fn=normalize_ref_text_with_steps,
        inputs=[ref_text_input],
        outputs=[ref_text_input, ref_norm_step1, ref_norm_step2]
    )

    # 🔴 ОБРАБОТЧИК ДЛЯ КНОПКИ НОРМАЛИЗАЦИИ ТЕКСТА ГЕНЕРАЦИИ
    normalize_gen_btn.click(
        fn=normalize_gen_text_with_steps,
        inputs=[gen_text_input],
        outputs=[gen_text_input, gen_norm_step1, gen_norm_step2]
    )

    with gr.Accordion("Advanced Settings", open=False):
        with gr.Row():
            # 🔴 ИСПРАВЛЕНИЕ: показываем реальный seed
            seed_input = gr.Number(label="Seed (random)", value=get_current_seed_display(), precision=0)
            remember_seed_checkbox = gr.Checkbox(label="💾 Запомнить этот seed", value=False)
            remove_silence = gr.Checkbox(label="Remove Silences", value=False)
        with gr.Row():
            speed_slider = gr.Slider(label="Speed", minimum=0.3, maximum=2.0, value=1.0, step=0.1)
            nfe_slider = gr.Slider(label="NFE Steps", minimum=4, maximum=64, value=48, step=2)
        with gr.Row():
            cross_fade_slider = gr.Slider(label="Cross-Fade Duration (s)", minimum=0.0, maximum=1.0, value=0.15, step=0.01)
            sway_sampling_slider = gr.Slider(label="Sway Sampling Coef", minimum=-1, maximum=1, value=-1, step=0.1)
        with gr.Row():
            cfg_strength_slider = gr.Slider(label="CFG Strength", minimum=0.5, maximum=5.0, value=2.0, step=0.1)
        with gr.Row():
            audio_format = gr.Radio(["wav", "mp3", "ogg", "flac"], label="Формат", value="wav")
            bitrate = gr.Radio(["128k", "192k", "320k"], label="Битрейт (mp3/ogg)", value="192k", visible=lambda fmt: fmt in ["mp3", "ogg"])

    # --- Обработчик изменения формата ---
    def update_bitrate_visibility(audio_format):
        return gr.update(visible=audio_format in ["mp3", "ogg"])

    audio_format.change(
        update_bitrate_visibility,
        inputs=audio_format,
        outputs=bitrate
    )

    # --- 🔴 КНОПКИ ГЕНЕРАЦИИ И ОСТАНОВКИ ---
    with gr.Row():
        generate_btn = gr.Button("🎤 Generate Speech", variant="primary", size="lg")
        stop_btn = gr.Button("🛑 Stop Generation", variant="stop", size="lg")

    with gr.Row():
        audio_output = gr.Audio(label="🎧 Аудио", type="filepath")
        spectrogram_output = gr.Image(label="Spectrogram", type="filepath")

    # --- 🔴 Обработчик остановки ---
    stop_btn.click(
        fn=stop_generation_process,
        outputs=None,
        queue=False
    )

    # 🔴 ИСПРАВЛЕННЫЙ ВЫЗОВ СИНТЕЗА С ОБНОВЛЕНИЕМ СТАТУСА
    generate_btn.click(
        synthesize,
        inputs=[
            ref_audio_input,
            ref_text_input,
            gen_text_input,
            remove_silence,
            seed_input,
            remember_seed_checkbox,
            model_choice,
            cross_fade_slider,
            nfe_slider,
            speed_slider,
            sway_sampling_slider,
            cfg_strength_slider,
            audio_format,
            bitrate,
        ],
        outputs=[
            audio_output, 
            spectrogram_output, 
            ref_text_input, 
            gen_text_input, 
            seed_input,
            model_status  # 🔴 ДОБАВЛЯЕМ ОБНОВЛЕНИЕ СТАТУСА МОДЕЛИ
        ]
    )

if __name__ == "__main__":
    # 🔴 ДОБАВЛЕНО: Проверяем модели перед запуском
    print("\n" + "="*60)
    print("🚀 ПОДГОТОВКА К ЗАПУСКУ")
    print("="*60)
    
    models_ok = check_all_models_before_launch()
    
    if models_ok:
        print("\n✅ Все модели проверены, запускаем интерфейс...")
        app.launch(share=True)
    else:
        print("\n⚠️ Некоторые модели не работают корректно")
        print("   Интерфейс будет запущен с ограниченной функциональностью")
        app.launch(share=True)


Checking CUDA availability...
CUDA is available. Using device: Tesla T4
Preloading model...
CUDA is available. Using device: Tesla T4
Trying to download model file 'espeech_tts_rlv2.pt' and 'vocab.txt' from hub 'ESpeech/ESpeech-TTS-1_RL-V2'


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Downloaded model to /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt
Downloaded vocab to /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/vocab.txt
Loading model from: /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt

vocab :  /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt 

Model loaded and moved to CUDA: cuda
Model preloaded.
Loading RUAccent...
RUAccent loaded.
Preloading vocoder...
CUDA is available. Using device: Tesla T4
Loading vocoder...
Download Vocos from huggingface charactr/vocos-mel-24khz
Vocoder